In [ ]:
import jax
from jax import tree_util
import pickle

def inspect_parameters(params):
    # 打印整体参数数量
    print(f"总参数数量: {jax.tree_util.tree_reduce(lambda x, y: x + y.size, params, 0)}")
    
    # 打印参数树的结构
    print("\n参数树结构:")
    jax.tree_util.tree_map(lambda x: print(f"形状: {x.shape}, 类型: {x.dtype}"), params)
    
    # 更详细的层次化参数结构
    flat_params = jax.tree_util.tree_flatten(params)[0]
    print(f"\n共有 {len(flat_params)} 个参数张量")
    
    for i, param in enumerate(flat_params):
        print(f"参数 {i}: 形状 {param.shape}, 大小 {param.size}, 类型 {param.dtype}")
        # 打印参数统计信息
        print(f"  - 均值: {param.mean():.6f}, 标准差: {param.std():.6f}")
        print(f"  - 最小值: {param.min():.6f}, 最大值: {param.max():.6f}")
        
# 方法2：从保存的pickle文件加载
with open("results.pkl", "rb") as f:
    saved_data = pickle.load(f)
    
# 检查加载的参数
inspect_parameters(saved_data["final_parameters"])

: 

In [ ]:
def analyze_vit_attention_weights(params):
    """分析ViT模型中的注意力权重"""
    # 找出所有注意力矩阵J
    attention_weights = {}
    
    def find_attention_matrices(params, path=''):
        if isinstance(params, dict):
            for key, value in params.items():
                # 检查是否是注意力层的J矩阵
                if key == 'J' and path.find('attn') != -1:
                    attention_weights[path] = value
                find_attention_matrices(value, f"{path}/{key}" if path else key)
    
    find_attention_matrices(params)
    
    # 可视化几个注意力头的权重
    for i, (path, weights) in enumerate(attention_weights.items()):
        if i >= 3:  # 仅展示前3个注意力层
            break
        
        plt.figure(figsize=(12, 4))
        plt.suptitle(f"注意力权重: {path}")
        
        if len(weights.shape) == 3:  # [头数, L_eff, L_eff]
            for h in range(min(3, weights.shape[0])):
                plt.subplot(1, 3, h+1)
                plt.imshow(weights[h], cmap='viridis')
                plt.colorbar()
                plt.title(f'头 {h}')
        
        plt.tight_layout()
        plt.savefig(f'attention_weights_{i}.png', dpi=300)
        plt.show()



2025-03-20 05:23:34.249852: W external/xla/xla/stream_executor/cuda/cuda_command_buffer.cc:662] Retry CUDA graph instantiation after OOM error
E0320 05:23:34.249983 2698845 pjrt_stream_executor_client.cc:3086] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Underlying backend ran out of memory trying to instantiate graph with 48 nodes and 0 conditionals (total of 0 alive graphs in the process). You can try to (a) Give more memory to the driver by reducing XLA_CLIENT_MEM_FRACTION (b) Disable command buffers with 'XLA_FLAGS=--xla_gpu_enable_command_buffer=' (empty set). Original error: Failed to instantiate CUDA graph: CUDA_ERROR_OUT_OF_MEMORY: out of memory


XlaRuntimeError: RESOURCE_EXHAUSTED: Underlying backend ran out of memory trying to instantiate graph with 48 nodes and 0 conditionals (total of 0 alive graphs in the process). You can try to (a) Give more memory to the driver by reducing XLA_CLIENT_MEM_FRACTION (b) Disable command buffers with 'XLA_FLAGS=--xla_gpu_enable_command_buffer=' (empty set). Original error: Failed to instantiate CUDA graph: CUDA_ERROR_OUT_OF_MEMORY: out of memory